In [2]:
import requests
from datetime import datetime
import pandas as pd
df=pd.read_csv('df11.csv')
import json
def get_codeforces_rating_history(username):
    url = f"https://codeforces.com/api/user.rating?handle={username}"
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            data = response.json()
            if data["status"] == "OK":
                return data["result"]
            else:
                print(f"API Error: {data.get('comment', 'No specific error message provided')}")
        except json.JSONDecodeError:
            print("Invalid JSON response from the API")
    else:
        print(f"Failed to fetch rating history for {username}. Status code: {response.status_code}")

    return None

def days_to_cross_margin(username, target_lower, target_upper):
    rating_history = get_codeforces_rating_history(username)
    
    if rating_history is None:
        return None

    current_rating = None
    start_date = None
    end_date = None

    for entry in rating_history:
        rating = entry["newRating"]
        contest_time = entry["ratingUpdateTimeSeconds"]
        contest_date = datetime.fromtimestamp(contest_time)

        if rating <= target_lower:
            start_date = contest_date
        elif rating >= target_upper:
            end_date = contest_date
            break

    if start_date and end_date:
        days_taken = (end_date - start_date).days
        return days_taken
    else:
        print(f"{username} did not cross the rating margin within the specified range.")
        return None

def add_result_to_dataframe(df):
    df['Result'] = ""
    target_lower = 1200
    target_upper = 1400

    for index, row in df.iterrows():
        username = row['Handle']
        days_taken = days_to_cross_margin(username, target_lower, target_upper)
        
        if days_taken is not None:
            if 20 <= days_taken <= 40:
                df.at[index, 'Result'] = "Yes"
            else:
                df.at[index, 'Result'] = "No"
        else:
            df.at[index, 'Result'] = "N/A"

    return df

if __name__ == "__main__":
    
    df = add_result_to_dataframe(df)

    print(df)

Failed to fetch rating history for wimg6__. Status code: 503
Failed to fetch rating history for wuhuqifeiwuhuqifei. Status code: 503
Failed to fetch rating history for Ahmed_Abo-eldahab. Status code: 503
Failed to fetch rating history for David9. Status code: 503
Failed to fetch rating history for INMAK. Status code: 503
Failed to fetch rating history for MIng_Dexterous. Status code: 503
Failed to fetch rating history for Mayank_Pushpjeet. Status code: 503
ariloc did not cross the rating margin within the specified range.
Failed to fetch rating history for champulal. Status code: 503
Failed to fetch rating history for creating001. Status code: 503
Failed to fetch rating history for daibowen. Status code: 503
Failed to fetch rating history for fanjige. Status code: 503
pandrei did not cross the rating margin within the specified range.
Failed to fetch rating history for roctes7. Status code: 503
Failed to fetch rating history for Hectorungo_18. Status code: 503
Failed to fetch rating hi

In [3]:
df.head()

,Handle,Max Rating,Result
0,wimg6__,1629,N/A
1,wuhuqifeiwuhuqifei,1629,N/A
2,Ahmed_Abo-eldahab,1682,N/A
3,Alphx,1705,No
4,David9,1732,N/A


In [4]:
df.to_csv('outdf11.csv', index=False)